In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Load data starting from 2025-02-01
start_date = '2025-02-01'
end_date = datetime.now().strftime('%Y-%m-%d')

print(f'Data range: {start_date} to {end_date}')

In [ ]:
# Filter data from 2025-02-01 onwards
df_filtered = df[df['date'] >= '2025-02-01']
print(f'Rows after filtering from 2025-02-01: {len(df_filtered)}')

In [ ]:
# Analysis from 2025-02-01
analysis_start = pd.to_datetime('2025-02-01')
iv_vs_rv = df[df['date'] >= '2025-02-01'].groupby('date').agg({
    'IV': 'mean',
    'RV': 'mean'
})
print(iv_vs_rv.head())